# Math Tutor Assistant

**NOTE:** This notebook is copied from [here](https://github.com/Azure/AI-in-a-Box/tree/main/gen-ai/Assistants/notebooks/math_tutor) and slightly modified to use the same environment variables as the previous labs. The [AI-In-a-Box repo](https://github.com/Azure/AI-in-a-Box) a curated collection of solution accelerators that can help engineers establish their AI/ML environments and solutions rapidly and with minimal friction, while maintaining the highest standards of quality and efficiency. Strongly recommended.

## Objective

This notebook showcases the foundational concepts of Assistants such as Threads, Messages, Runs, Tools, and lifecycle management.

Reference:
- Learn more about how to use Assistants with our [How-to guide on Assistants](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/assistant)
- [Assistants OpenAI Overview](https://platform.openai.com/docs/assistants/overview)
- [Github-OpenAI Python/examples/Assistant demo notebook](https://github.com/openai/openai-python/blob/main/examples/assistant.py)

## About this example

This sample shows users how to create an Azure OpenAI Assistant named "Math Tutor" using the Azure OpenAI API. The assistant is designed to function as a personal math tutor, capable of answering math questions through code interpretation. The script initiates a conversation with the assistant, guiding it through various mathematical queries and scenarios to showcase its capabilities.

This sample provides developers with a clear demonstration of how to leverage the core concepts of the Assistants API into their projects, highlighting its simplicity and effectiveness in leveraging foundational concepts.

### Parameters

In [6]:
import os

from dotenv import load_dotenv

load_dotenv("../.env")  # make sure to have the .env file in the root directory of the project

api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = "2024-05-01-preview"
api_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_ID")

should_cleanup: bool = True

print('Using API version ' + api_version)
print('Using API deployment ' + api_deployment_name)

Using API version 2024-02-01
Using API deployment gpt-35-turbo-1106


## Run this Example

### Load the required libraries

In [2]:
import io
import time
from datetime import datetime
from typing import Iterable

from openai import AzureOpenAI
from openai.types.beta.threads.text_content_block import TextContentBlock
from openai.types.beta.threads.image_file_content_block import ImageFileContentBlock
from PIL import Image

### Create an Azure OpenAI client

In [3]:
client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint)

### Create an Assistant and a Thread

In [4]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model=api_deployment_name,
)

thread = client.beta.threads.create()

NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}

### Format and display the Assistant Messages for text and images

In [ ]:
def read_assistant_file(file_id:str):
    response_content = client.files.content(file_id)
    return response_content.read()

def print_messages(messages) -> None:
    message_list = []

    # Get all the messages till the last user message
    for message in messages:
        message_list.append(message)
        if message.role == "user":
            break

    # Reverse the messages to show the last user message first
    message_list.reverse()

    # Print the user or Assistant messages or images
    for message in message_list:
        for item in message.content:
            # Determine the content type
            if isinstance(item, TextContentBlock):
                print(f"{message.role}:\n{item.text.value}\n")
                file_annotations = item.text.annotations
                if file_annotations:
                    for annotation in file_annotations:
                        file_id = annotation.file_path.file_id
                        content = read_assistant_file(file_id)
                        print(f"Annotation Content:\n{str(content)}\n")
            elif isinstance(item, ImageFileContentBlock):
                # Retrieve image from file id                
                data_in_bytes = read_assistant_file(item.image_file.file_id)
                # Convert bytes to image
                readable_buffer = io.BytesIO(data_in_bytes)
                image = Image.open(readable_buffer)
                # Resize image to fit in terminal
                width, height = image.size
                image = image.resize((width // 2, height // 2), Image.LANCZOS)
                # Display image
                image.show()

### Process the user messages

In [ ]:
def process_prompt(prompt: str) -> None:
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=prompt)
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="Please address the user as Jane Doe. The user has a premium account. Be assertive, accurate, and polite. Ask if the user has further questions. "
        + "The current date and time is: "
        + datetime.now().strftime("%x %X")
        + ". ",
    )
    print("processing ...")
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run.status == "completed":
            # Handle completed
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            print_messages(messages)
            break
        if run.status == "failed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            answer = messages.data[0].content[0].text.value
            print(f"Failed User:\n{prompt}\nAssistant:\n{answer}\n")
            # Handle failed
            break
        if run.status == "expired":
            # Handle expired
            print(run)
            break
        if run.status == "cancelled":
            # Handle cancelled
            print(run)
            break
        if run.status == "requires_action":
            # Handle function calling and continue processing
            pass
        else:
            time.sleep(5)

### Have a conversation with the Assistant

In [ ]:
process_prompt("What is the linear equation when two (x,y) points are (1,1) and (5,10)?")

In [ ]:
process_prompt("I need to solve the equation `3x + 11 = 14`. Can you help me?")

In [ ]:
process_prompt("""x=r*cos(u)sin(v), y=r*sin(u)sin(v), r=2+sin(7*u+5*v) for 0<u<2π and 0<v<π.
Create a graph of the equation z=r*cos(v).""")

In [ ]:
process_prompt("create a csv file with 10 customer names")

### Cleaning up

In [ ]:
if should_cleanup:
    client.beta.assistants.delete(assistant.id)
    client.beta.threads.delete(thread.id)